In [201]:
with open('stripe.txt') as file:
    transactions_text = file.read()

In [202]:
def calculate_transactions_info(transactions, start_timestamp, end_timestamp):
    # Convert start and end timestamps to datetime objects
    start_time = datetime.fromisoformat(start_timestamp)
    end_time = datetime.fromisoformat(end_timestamp)
    
    # Initialize counters
    num_transactions = 0
    total_amount = 0.0
    
    # Iterate over transactions and calculate info within the specified range
    for timestamp, amount in transactions.items():
        transaction_time = datetime.fromisoformat(timestamp)
        
        if start_time <= transaction_time < end_time:
            num_transactions += 1
            total_amount += amount
    
    return num_transactions, total_amount


In [203]:
import re
import pytz
from datetime import datetime, timedelta

def convert_transactions_json(transactions_text):
    # Define the EDT timezone
    edt = pytz.timezone('America/New_York')

    # Define the regular expression pattern to match the entire transaction
    pattern = r'Stripe\nBOT\n — (.*(?:AM|PM))\nCha-ching! Someone just bought \$([\d.]+) of'

    # Find all matches of the pattern in transactions_text
    matches = re.findall(pattern, transactions_text)

    # Create a dictionary to store the transactions
    transactions = {}

    # Iterate over matches and store in the dictionary
    for match in matches:
        raw_time = match[0]
        amount = float(match[1])

        if "Yesterday" in raw_time:
            # Handle Yesterday case
            yesterday = datetime.now(edt) - timedelta(days=1)
            raw_time = raw_time.replace("Yesterday", yesterday.strftime("%m/%d/%Y"))

        if "Today" in raw_time:
            # Handle Today case
            today = datetime.now(edt)
            raw_time = raw_time.replace("Today", today.strftime("%m/%d/%Y"))

        raw_time = raw_time.replace(" at ", " ")

        # Convert the raw time to a datetime object in EDT timezone
        transaction_time = edt.localize(datetime.strptime(raw_time, "%m/%d/%Y %I:%M %p"))

        # Store the transaction using ISO datestamp with timezone as key
        transactions[transaction_time.isoformat()] = amount
        
    return transactions

In [204]:
transactions = convert_transactions_json(transactions_text)

# Example usage
start_timestamp = "2023-08-04T00:00:00-08:00"  # Replace with your desired start timestamp
end_timestamp = "2023-08-05T00:00:00-08:00"    # Replace with your desired end timestamp

num_transactions, total_amount = calculate_transactions_info(transactions, start_timestamp, end_timestamp)

print("Number of transactions:", num_transactions)
print("Total amount:", total_amount)

Number of transactions: 13
Total amount: 1285.0


In [205]:

start = "2023-07-01T00:00:00-08:00"
num_days = 31  # Change this value to the desired number of days to look over

# Parse the string into a datetime object
start_datetime = datetime.fromisoformat(start)
# Iterate over the range of num_days and print the string for each day
range_total_amount = 0
range_total_transactions = 0
range_amount_numbers = []
for day in range(num_days):
    # Add 'day' number of days to the datetime object
    day_start_datetime = start_datetime + timedelta(days=day)
    day_end_datetime = start_datetime + timedelta(days=day+1)
    start_timestamp = datetime.isoformat(day_start_datetime)
    end_timestamp = datetime.isoformat(day_end_datetime)
    num_transactions, total_amount = calculate_transactions_info(transactions, start_timestamp, end_timestamp)
    print(f"Day: {start_timestamp} to {end_timestamp}")
    print("Number of transactions:", num_transactions)
    print("Total amount:", total_amount)
    print("")
    range_total_transactions += num_transactions
    range_total_amount += total_amount
    range_amount_numbers.append(total_amount)
end_datetime = datetime.fromisoformat(start) + timedelta(days=num_days)
print("")
print(f"Range: {start_datetime} to {end_datetime}")
print(f"# transactions in range: {range_total_transactions}")
print(f"$ total in range: {range_total_amount}")

daily_revenue_july_calculated_from_bot = range_amount_numbers

Day: 2023-07-01T00:00:00-08:00 to 2023-07-02T00:00:00-08:00
Number of transactions: 7
Total amount: 145.0

Day: 2023-07-02T00:00:00-08:00 to 2023-07-03T00:00:00-08:00
Number of transactions: 5
Total amount: 155.0

Day: 2023-07-03T00:00:00-08:00 to 2023-07-04T00:00:00-08:00
Number of transactions: 1
Total amount: 25.0

Day: 2023-07-04T00:00:00-08:00 to 2023-07-05T00:00:00-08:00
Number of transactions: 1
Total amount: 10.0

Day: 2023-07-05T00:00:00-08:00 to 2023-07-06T00:00:00-08:00
Number of transactions: 3
Total amount: 60.0

Day: 2023-07-06T00:00:00-08:00 to 2023-07-07T00:00:00-08:00
Number of transactions: 4
Total amount: 70.0

Day: 2023-07-07T00:00:00-08:00 to 2023-07-08T00:00:00-08:00
Number of transactions: 3
Total amount: 60.0

Day: 2023-07-08T00:00:00-08:00 to 2023-07-09T00:00:00-08:00
Number of transactions: 1
Total amount: 25.0

Day: 2023-07-09T00:00:00-08:00 to 2023-07-10T00:00:00-08:00
Number of transactions: 5
Total amount: 110.0

Day: 2023-07-10T00:00:00-08:00 to 2023-07-1

In [228]:
import numpy as np
# July 2023 numbers actual (from stats page)
daily_revenue_july_from_stats_page = [170,
155,
25,
10,
60,
70,
60,
25,
110,
70,
40,
10,
10,
35,
25,
10,
150,
0,
120,
120,
120,
0,
0,
110,
225,
305,
590,
330,
20,
320,
2975]

sum_stats = np.sum(daily_revenue_july_from_stats_page)
print(f"July revenue total (reported by stats page): {sum_stats:0.1f}")
sum_bot = np.sum(daily_revenue_july_calculated_from_bot)
print(f"July revenue total (calculated from bot): {sum_bot:0.1f}")
print("")
print("Days with differences in revenue (Bot - Stats page):")
diff_total = 0
for i in range(0, 31):
    diff = daily_revenue_july_calculated_from_bot[i] - daily_revenue_july_from_stats_page[i]
    if diff != 0:
        print(f"  July {i+1:2d}: {diff:0.1f}")
    diff_total += diff
print(f"Differences in total revenue (Bot - Stats page): {diff_total:0.1f}")

July revenue total (reported by stats page): 6270.0
July revenue total (calculated from bot): 6905.0

Days with differences in revenue (Bot - Stats page):
  July  1: -25.0
  July 28: 220.0
  July 31: 440.0
Differences in total revenue (Bot - Stats page): 635.0
